In [1]:
import numpy as np

In [2]:
y_train = np.load('../benchmark/RAW_DATA/PTBXL/y_train.npy')

In [3]:
y_train.shape

(19634, 71)

In [4]:
sample_positive_count = y_train.sum(axis=1)
sample_positive_count.shape

(19634,)

In [5]:
sample_positive_count.min(), sample_positive_count.max()

(1, 9)

In [6]:
label_positive_count = y_train.sum(axis=0)
label_positive_count.shape

(71,)

In [7]:
label_positive_count[np.argsort(label_positive_count)]

array([    9,    13,    13,    14,    15,    15,    16,    18,    22,
          24,    25,    27,    30,    32,    40,    46,    56,    66,
          69,    72,    74,    88,    89,    94,   107,   114,   128,
         134,   143,   153,   159,   161,   163,   164,   181,   193,
         197,   261,   265,   267,   306,   319,   358,   382,   385,
         394,   431,   482,   488,   493,   573,   594,   693,   695,
         710,   717,   744,   788,   908,  1006,  1031,  1147,  1362,
        1463,  1646,  1923,  2127,  2417,  3005,  8564, 15104])

In [8]:
np.argsort(label_positive_count)

array([52,  1, 21,  2, 51, 23, 22, 68, 53, 66, 64, 60, 26, 67, 29, 25, 15,
        5, 16, 70, 10, 14, 56,  8, 39, 57, 33, 19, 65, 30, 41, 31, 13, 44,
       38, 20, 32,  6, 24, 50, 42,  7, 49, 48, 37, 40, 17, 11, 12, 55, 59,
       28, 47, 58, 35,  0, 62, 69, 63, 27, 54, 34,  4, 36, 45, 43,  9, 18,
        3, 46, 61])

In [9]:
count = 0
i = 0
indices = list()
all = list(range(y_train.shape[0]))
while len(all) > 0:
    current_label = y_train[:, np.argsort(label_positive_count)[i]]
    current_positive = np.where(current_label == 1.0)[0].tolist()
    choices = set(current_positive).intersection(set(all))
    if len(choices) > 0:
        indices.append(list(choices))
        all = [index for index in all if index not in choices]
        count += 1
    i += 1
count, i

(70, 70)

In [10]:
y_train[list(choices), 46].sum()

6564

In [11]:
from matplotlib import pyplot as plt

In [12]:
import json

In [13]:
data = json.load(open('../fedtask/ptbxl_cnum1_iid_0+1/data.json', 'r'))

In [14]:
data.keys()

dict_keys(['store', 'dtest', 'datasrc', 'client_names', 'Client0'])

In [15]:
data['Client0']['modalities']

'all'

In [16]:
new_data = {
    'store': data['store'],
    'dtest': data['dtest'],
    'datasrc': data['datasrc'],
    'client_names': ['Client{:02d}'.format(i) for i in range(100)]
}

In [17]:
for i, choices in enumerate(indices[:-1]):
    new_data['Client{:02d}'.format(i)] = {
        'dtrain': choices,
        'dvalid': list(),
        'modalities': 'all'
    }

In [18]:
import random

In [19]:
random.seed(369)
np.random.seed(369)
pvals = np.random.dirichlet(alpha=[1.0] * 31)
split_lengths = np.random.multinomial(len(indices[-1]), pvals=pvals)
split_lengths

array([165,  83, 105,  96, 299,  20,  91, 141,  22,  58,  48, 478, 152,
       233, 143,  30, 552,  20, 383, 163, 499, 157, 123,  54, 443,  66,
       238, 708, 261, 692,  41])

In [20]:
choices = indices[-1]
random.shuffle(choices)
start = 0
for i, length in enumerate(split_lengths):
    new_data['Client{:02d}'.format(i + 69)] = {
        'dtrain': choices[start:start + length],
        'dvalid': list(),
        'modalities': 'all'
    }
    start += length

In [21]:
with open('../fedtask/ptbxl_cnum100_non_iid_0+100/data.json', 'w') as f:
    json.dump(new_data, f)